In [ ]:
from pyBADA.bada4 import Bada4Aircraft, BADA4
from pyBADA.aircraft import Airplane
import pyBADA.atmosphere as atm
import math
import pandas as pd
import numpy as np

# 自定义转换函数
class conv:
    @staticmethod
    def ft2m(feet):
        return feet * 0.3048
    
    @staticmethod
    def m2ft(meters):
        return meters / 0.3048
    
    @staticmethod
    def kt2ms(knots):
        return knots * 0.51444
    
    @staticmethod
    def ms2kt(ms):
        return ms / 0.51444

def calculate_descent_profile_detailed(cruise_fl, target_fl, aircraft_mass, 
                                     descent_mach, high_cas, ac_model, 
                                     low_cas=None, low_cas_fl=100, 
                                     constant_cas=None, print_details=True):
    """
    计算下降剖面并打印详细信息
    
    参数:
    cruise_fl: 巡航高度层 (FL)
    target_fl: 目标高度层 (FL)
    aircraft_mass: 飞机重量 (kg)
    descent_mach: 下降马赫数，如果constant_cas不为None则忽略
    high_cas: 过渡高度后的CAS (kt)，如果constant_cas不为None则忽略
    ac_model: 飞机型号 (必需参数)
    low_cas: 低高度CAS (kt)，如果为None则与high_cas相同
    low_cas_fl: 低高度CAS开始的高度层 (FL)
    constant_cas: 如果不为None，则使用恒定CAS下降
    print_details: 是否打印详细信息
    
    返回:
    包含下降剖面总结信息的字典和详细DataFrame
    """
    # 固定的BADA配置
    bada_version = "4.2"
    filepath = "/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"
    
    # 初始化飞机模型
    AC = Bada4Aircraft(
        badaVersion=bada_version,
        acName=ac_model,
        filePath=filepath,
    )
    
    # 创建BADA4性能计算对象
    bada4 = BADA4(AC)
    
    # 设置低高度CAS
    if low_cas is None:
        low_cas = high_cas
    
    # 为恒定CAS下降设置参数
    if constant_cas is not None:
        descent_mach = 0.9  # 设置一个较高的马赫数，确保过渡高度远高于巡航高度
        high_cas = constant_cas
        low_cas = constant_cas if low_cas is None else low_cas
    
    # 计算过渡高度
    high_cas_ms = conv.kt2ms(high_cas)
    crossover_m = atm.crossOver(cas=high_cas_ms, Mach=descent_mach)
    crossover_ft = conv.m2ft(crossover_m)
    crossover_fl = int(crossover_ft / 100)
    
    # 创建速度剖面名称
    if constant_cas is not None:
        if high_cas == low_cas:
            profile_name = f"{high_cas}kt constant"
        else:
            profile_name = f"{high_cas}kt→{low_cas}kt@FL{low_cas_fl}"
    else:
        if high_cas == low_cas:
            profile_name = f"{descent_mach}M/{high_cas}kt"
        else:
            profile_name = f"{descent_mach}M/{high_cas}kt/{low_cas}kt@FL{low_cas_fl}"
    
    if print_details:
        print(f"\n{'='*80}")
        print(f"详细下降剖面分析: {ac_model}")
        print(f"{'='*80}")
        print(f"飞机重量: {aircraft_mass/1000:.1f} 吨")
        print(f"下降剖面: {profile_name}")
        print(f"巡航高度: FL{cruise_fl} ({cruise_fl*100:,} ft)")
        print(f"目标高度: FL{target_fl} ({target_fl*100:,} ft)")
        print(f"过渡高度: FL{crossover_fl} ({crossover_fl*100:,} ft)")
        print(f"{'='*80}")
    
    def calculate_performance(flight_level, aircraft_mass, DeltaTemp=0):
        """计算指定高度层的下降性能"""
        altitude_ft = flight_level * 100
        altitude_m = conv.ft2m(altitude_ft)
        
        # 计算大气参数
        theta_val, delta_val, sigma_val = atm.atmosphereProperties(altitude_m, DeltaTemp)
        
        # 根据下降速度规则确定速度
        if constant_cas is None and flight_level >= crossover_fl:
            # 使用指定马赫数
            M = descent_mach
            tas = atm.mach2Tas(M, theta_val)
            cas = atm.mach2Cas(M, theta_val, delta_val, sigma_val)
            speed_mode = f"Mach {descent_mach}"
            flight_evolution = "constM"
        elif flight_level >= low_cas_fl:
            # 使用高高度CAS
            cas = conv.kt2ms(high_cas)
            tas = atm.cas2Tas(cas, delta_val, sigma_val)
            M = atm.tas2Mach(tas, theta_val)
            speed_mode = f"CAS {high_cas}kt"
            flight_evolution = "constCAS"
        else:
            # 使用低高度CAS
            cas = conv.kt2ms(low_cas)
            tas = atm.cas2Tas(cas, delta_val, sigma_val)
            M = atm.tas2Mach(tas, theta_val)
            speed_mode = f"CAS {low_cas}kt"
            flight_evolution = "constCAS"
        
        # 动态计算能量分配因子
        esf_des = Airplane.esf(
            h=altitude_m,
            DeltaTemp=DeltaTemp,
            flightEvolution=flight_evolution,
            phase="des",
            M=M
        )
        
        # 计算升力系数
        cl = bada4.CL(delta=delta_val, mass=aircraft_mass, M=M)
        
        # 计算阻力系数(干净构型)
        HLid = 0.0  # 无襟翼
        LG = "LGUP"  # 起落架收起
        cd = bada4.CD(HLid=HLid, LG=LG, CL=cl, M=M)
        
        # 计算阻力
        drag = bada4.D(delta=delta_val, M=M, CD=cd)
        
        # 计算怠速推力
        idle_thrust = bada4.Thrust(
            delta=delta_val,
            theta=theta_val,
            M=M,
            rating="LIDL",  # 怠速设置
            DeltaTemp=DeltaTemp
        )
        
        # 计算燃油流量 (kg/s)
        fuel_flow = bada4.ff(
            delta=delta_val,
            theta=theta_val,
            M=M,
            rating="LIDL",  # 怠速设置
            DeltaTemp=DeltaTemp
        )
        
        # 转换为kg/h
        fuel_flow_kgh = fuel_flow * 3600
        
        # 计算怠速下降率
        idle_descent_rate = bada4.ROCD(
            T=idle_thrust,
            D=drag,
            v=tas,
            mass=aircraft_mass,
            ESF=esf_des,
            h=altitude_m,
            DeltaTemp=DeltaTemp
        )
        
        # 计算下降角(度)
        if tas > 0:
            descent_angle = math.degrees(math.asin(idle_descent_rate / tas))
        else:
            descent_angle = 0
        
        # 计算下降梯度(百分比)
        descent_gradient = 100 * abs(idle_descent_rate / tas) if tas > 0 else 0
        
        # 计算高距离
        ft_per_nm = abs(idle_descent_rate * 196.85 / conv.ms2kt(tas) * 60) if conv.ms2kt(tas) > 0 else 0
        
        return {
            "FL": flight_level,
            "高度(ft)": altitude_ft,
            "速度模式": speed_mode,
            "马赫数": round(M, 3),
            "CAS(kt)": round(conv.ms2kt(cas), 1),
            "TAS(kt)": round(conv.ms2kt(tas), 1),
            "下降率(ft/min)": round(idle_descent_rate * 196.85, 0),
            "下降率(m/s)": round(idle_descent_rate, 2),
            "下降角(度)": round(descent_angle, 2),
            "下降梯度(%)": round(descent_gradient, 2),
            "高距离(ft/nm)": round(ft_per_nm, 0),
            "ESF": round(esf_des, 3),
            "阻力(N)": round(drag, 0),
            "怠速推力(N)": round(idle_thrust, 0),
            "燃油流量(kg/h)": round(fuel_flow_kgh, 1),
            "燃油流量(kg/s)": round(fuel_flow, 4),
            "升力系数": round(cl, 3),
            "阻力系数": round(cd, 4)
        }
    
    # 高度采样 - 每10个高度层一个点，提供更详细的数据
    flight_levels = []
    for fl in range(cruise_fl, target_fl-1, -10):
        flight_levels.append(fl)
    if target_fl not in flight_levels:
        flight_levels.append(target_fl)
    
    results = []
    for fl in flight_levels:
        result = calculate_performance(fl, aircraft_mass)
        results.append(result)
    
    # 创建DataFrame展示结果
    df = pd.DataFrame(results)
    
    # 计算累积距离和燃油消耗
    horizontal_distance = np.zeros(len(flight_levels))
    fuel_consumption = np.zeros(len(flight_levels))
    time_elapsed = np.zeros(len(flight_levels))
    
    for i in range(1, len(flight_levels)):
        # 计算相邻高度层之间的高度差(英尺)
        altitude_diff = (flight_levels[i-1] - flight_levels[i]) * 100
        # 使用两个点的平均高距离比
        avg_ft_per_nm = (df.iloc[i-1]['高距离(ft/nm)'] + df.iloc[i]['高距离(ft/nm)']) / 2
        if avg_ft_per_nm > 0:
            segment_distance = altitude_diff / avg_ft_per_nm
        else:
            segment_distance = 0
        # 累积水平距离
        horizontal_distance[i] = horizontal_distance[i-1] + segment_distance
        
        # 计算时间和燃油
        avg_tas_per_sec = (df.iloc[i-1]['TAS(kt)'] + df.iloc[i]['TAS(kt)']) / (2 * 3600)
        if avg_tas_per_sec > 0:
            segment_time_seconds = segment_distance / avg_tas_per_sec
        else:
            segment_time_seconds = 0
        
        time_elapsed[i] = time_elapsed[i-1] + segment_time_seconds
        
        avg_fuel_flow = (df.iloc[i-1]['燃油流量(kg/s)'] + df.iloc[i]['燃油流量(kg/s)']) / 2
        segment_fuel = avg_fuel_flow * segment_time_seconds
        fuel_consumption[i] = fuel_consumption[i-1] + segment_fuel
    
    # 添加累积数据到DataFrame
    df['累积距离(nm)'] = [round(d, 1) for d in horizontal_distance]
    df['累积时间(min)'] = [round(t/60, 1) for t in time_elapsed]
    df['累积燃油(kg)'] = [round(f, 1) for f in fuel_consumption]
    
    if print_details:
        # 打印详细表格
        print("\n详细下降数据:")
        print("-" * 150)
        
        # 选择要显示的关键列
        display_columns = [
            'FL', '速度模式', '马赫数', 'CAS(kt)', 'TAS(kt)', 
            '下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)',
            '燃油流量(kg/h)', '累积距离(nm)', '累积时间(min)', '累积燃油(kg)'
        ]
        
        # 打印表头
        header = ""
        for col in display_columns:
            if col in ['FL', '马赫数', 'CAS(kt)', 'TAS(kt)']:
                header += f"{col:>8}"
            elif col in ['速度模式']:
                header += f"{col:>12}"
            elif col in ['下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)', '燃油流量(kg/h)']:
                header += f"{col:>12}"
            else:
                header += f"{col:>14}"
        print(header)
        print("-" * 150)
        
        # 打印数据行
        for _, row in df.iterrows():
            line = ""
            for col in display_columns:
                value = row[col]
                if col in ['FL']:
                    line += f"{value:>8.0f}"
                elif col in ['马赫数']:
                    line += f"{value:>8.3f}"
                elif col in ['CAS(kt)', 'TAS(kt)']:
                    line += f"{value:>8.1f}"
                elif col in ['速度模式']:
                    line += f"{str(value):>12}"
                elif col in ['下降率(ft/min)', '高距离(ft/nm)']:
                    line += f"{value:>12.0f}"
                elif col in ['下降梯度(%)']:
                    line += f"{value:>12.2f}"
                elif col in ['燃油流量(kg/h)']:
                    line += f"{value:>12.1f}"
                else:
                    line += f"{value:>14.1f}"
            print(line)
        
        print("-" * 150)
    
    # 计算总结信息
    total_distance = horizontal_distance[-1]
    total_time = time_elapsed[-1]
    total_fuel = fuel_consumption[-1]
    total_altitude_change = (cruise_fl - target_fl) * 100
    avg_ft_per_nm = total_altitude_change / total_distance if total_distance > 0 else 0
    
    summary = {
        "Profile": profile_name,
        "Descent Distance (nm)": round(total_distance, 1),
        "Descent Time (s)": round(total_time, 0),
        "Descent Time (min)": round(total_time/60, 1),
        "Fuel Consumption (kg)": round(total_fuel, 1),
        "Average Fuel Flow (kg/h)": round(total_fuel/(total_time/3600), 1) if total_time > 0 else 0,
        "Average Descent Gradient (%)": round(total_altitude_change/6076.12/total_distance*100, 2) if total_distance > 0 else 0,
        "Altitude-to-Distance Ratio (ft/nm)": round(avg_ft_per_nm, 1)
    }
    
    if print_details:
        print(f"\n下降剖面总结:")
        print(f"总下降距离: {summary['Descent Distance (nm)']} nm")
        print(f"总下降时间: {summary['Descent Time (min)']} 分钟")
        print(f"总燃油消耗: {summary['Fuel Consumption (kg)']} kg")
        print(f"平均燃油流量: {summary['Average Fuel Flow (kg/h)']} kg/h")
        print(f"平均下降梯度: {summary['Average Descent Gradient (%)']} %")
        print(f"高距比: {summary['Altitude-to-Distance Ratio (ft/nm)']} ft/nm")
        print("=" * 80)
    
    return summary, df

# ==============================================================================
# 主程序：分析指定的下降剖面
# ==============================================================================

print("开始分析下降剖面...")

# 主要分析：0.73M/250kt/220kt@FL150 剖面
result_summary, result_df = calculate_descent_profile_detailed(
    cruise_fl=370,
    target_fl=30,
    aircraft_mass=60000,
    descent_mach=0.73,
    high_cas=250,
    ac_model="A320-232",
    low_cas=220,
    low_cas_fl=150,
    print_details=True
)

print("\n" + "="*80)
print("其他下降剖面示例:")
print("="*80)

# 示例1: 没有low_cas的情况（全程使用相同CAS）
print("\n示例1: 标准下降剖面 (0.73M/250kt)")
result1_summary, result1_df = calculate_descent_profile_detailed(
    cruise_fl=370,
    target_fl=30,
    aircraft_mass=60000,
    descent_mach=0.73,
    high_cas=250,
    ac_model="A320-232",
    print_details=True
)

# 示例2: 恒定CAS下降
print("\n示例2: 恒定CAS下降 (230kt)")
result2_summary, result2_df = calculate_descent_profile_detailed(
    cruise_fl=370,
    target_fl=30,
    aircraft_mass=60000,
    descent_mach=0.90,  # 这个参数在constant_cas模式下不会被使用
    high_cas=230,  # 这个参数在constant_cas模式下不会被使用
    ac_model="A320-232",
    constant_cas=230,
    print_details=True
)

开始分析下降剖面...

详细下降剖面分析: A320-232
飞机重量: 60.0 吨
下降剖面: 0.73M/250kt/220kt@FL150
巡航高度: FL370 (37,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL342 (34,200 ft)

详细下降数据:
------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)     累积时间(min)      累积燃油(kg)
------------------------------------------------------------------------------------------------------------------------------------------------------
     370   Mach 0.73   0.730   234.7   418.7       -2234        5.27         320       535.0           0.0           0.0           0.0
     360   Mach 0.73   0.730   240.3   418.9       -2425        5.72         347       533.3           3.0           0.4           3.8
     350   Mach 0.73   0.730   245.9   420.8       -2475        5.81         353       531.4           5.9           0.8           7.4
    

In [4]:
print("\n示例2: 恒定CAS下降 (230kt)")
result2_summary, result2_df = calculate_descent_profile_detailed(
    cruise_fl=390,
    target_fl=30,
    aircraft_mass= 165000,    # MLW: 172,365   B788GE70
    descent_mach=0.86,  # 这个参数在0.90 constant_cas模式下不会被使用
    high_cas=330,  # 这个参数在constant_cas模式下不会被使用
    ac_model="B788GE70",
    low_cas=250,
    low_cas_fl=100,
    print_details=True
)


示例2: 恒定CAS下降 (230kt)

详细下降剖面分析: B788GE70
飞机重量: 165.0 吨
下降剖面: 0.86M/330kt/250kt@FL100
巡航高度: FL390 (39,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL298 (29,800 ft)

详细下降数据:
------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)     累积时间(min)      累积燃油(kg)
------------------------------------------------------------------------------------------------------------------------------------------------------
     390   Mach 0.86   0.860   268.9   493.3       -2496        5.00         304      1199.9           0.0           0.0           0.0
     380   Mach 0.86   0.860   275.2   493.3       -2573        5.15         313      1168.7           3.2           0.4           7.8
     370   Mach 0.86   0.860   281.6   493.3       -2663        5.33         324      1137.1           6.4           0.8        

In [7]:
from pyBADA.bada4 import Bada4Aircraft, BADA4
from pyBADA.aircraft import Airplane
import pyBADA.atmosphere as atm
import math
import pandas as pd
import numpy as np

# 自定义转换函数
class conv:
    @staticmethod
    def ft2m(feet):
        return feet * 0.3048
    
    @staticmethod
    def m2ft(meters):
        return meters / 0.3048
    
    @staticmethod
    def kt2ms(knots):
        return knots * 0.51444
    
    @staticmethod
    def ms2kt(ms):
        return ms / 0.51444
    
# 定义速度限制，格式为 [(飞行高度层, 最大允许CAS), ...]
speed_restrictions = [(100, 250)]  # FL100以下限制250节

# 定义减速段计算函数
def calculate_deceleration_segment(current_cas, target_cas, tas_kt, fuel_flow_kg_per_sec):
    """计算减速段所需的距离和时间"""
    if current_cas <= target_cas:
        return 0, 0, 0  # 已经低于目标速度，不需要减速
    
    # 使用经验公式
    decel_distance_nm = (current_cas - target_cas) / 10
    decel_time_sec = (current_cas - target_cas)
    
    # 使用当前高度的燃油流量计算减速段燃油消耗
    decel_fuel = fuel_flow_kg_per_sec * decel_time_sec
    
    return decel_distance_nm, decel_time_sec, decel_fuel


def calculate_descent_profile_detailed(cruise_fl, target_fl, aircraft_mass, 
                                     descent_mach, high_cas, ac_model, 
                                     low_cas=None, low_cas_fl=100, 
                                     constant_cas=None, print_details=True):
    """
    计算下降剖面并打印详细信息
    
    参数:
    cruise_fl: 巡航高度层 (FL)
    target_fl: 目标高度层 (FL)
    aircraft_mass: 飞机重量 (kg)
    descent_mach: 下降马赫数，如果constant_cas不为None则忽略
    high_cas: 过渡高度后的CAS (kt)，如果constant_cas不为None则忽略
    ac_model: 飞机型号 (必需参数)
    low_cas: 低高度CAS (kt)，如果为None则与high_cas相同
    low_cas_fl: 低高度CAS开始的高度层 (FL)
    constant_cas: 如果不为None，则使用恒定CAS下降
    print_details: 是否打印详细信息
    
    返回:
    包含下降剖面总结信息的字典和详细DataFrame
    """
    # 固定的BADA配置
    bada_version = "4.2"
    filepath = "/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"
    
    # 初始化飞机模型
    AC = Bada4Aircraft(
        badaVersion=bada_version,
        acName=ac_model,
        filePath=filepath,
    )
    
    # 创建BADA4性能计算对象
    bada4 = BADA4(AC)
    
    # 设置低高度CAS
    if low_cas is None:
        low_cas = high_cas
    
    # 为恒定CAS下降设置参数
    if constant_cas is not None:
        descent_mach = 0.9  # 设置一个较高的马赫数，确保过渡高度远高于巡航高度
        high_cas = constant_cas
        low_cas = constant_cas if low_cas is None else low_cas
    
    # 计算过渡高度
    high_cas_ms = conv.kt2ms(high_cas)
    crossover_m = atm.crossOver(cas=high_cas_ms, Mach=descent_mach)
    crossover_ft = conv.m2ft(crossover_m)
    crossover_fl = int(crossover_ft / 100)
    
    # 创建速度剖面名称
    if constant_cas is not None:
        if high_cas == low_cas:
            profile_name = f"{high_cas}kt constant"
        else:
            profile_name = f"{high_cas}kt→{low_cas}kt@FL{low_cas_fl}"
    else:
        if high_cas == low_cas:
            profile_name = f"{descent_mach}M/{high_cas}kt"
        else:
            profile_name = f"{descent_mach}M/{high_cas}kt/{low_cas}kt@FL{low_cas_fl}"
    
    if print_details:
        print(f"\n{'='*80}")
        print(f"详细下降剖面分析: {ac_model}")
        print(f"{'='*80}")
        print(f"飞机重量: {aircraft_mass/1000:.1f} 吨")
        print(f"下降剖面: {profile_name}")
        print(f"巡航高度: FL{cruise_fl} ({cruise_fl*100:,} ft)")
        print(f"目标高度: FL{target_fl} ({target_fl*100:,} ft)")
        print(f"过渡高度: FL{crossover_fl} ({crossover_fl*100:,} ft)")
        print(f"{'='*80}")
    
    def calculate_performance(flight_level, aircraft_mass, DeltaTemp=0):
        """计算指定高度层的下降性能"""
        altitude_ft = flight_level * 100
        altitude_m = conv.ft2m(altitude_ft)
        
        # 计算大气参数
        theta_val, delta_val, sigma_val = atm.atmosphereProperties(altitude_m, DeltaTemp)
        
        # 根据下降速度规则确定速度
        if constant_cas is None and flight_level >= crossover_fl:
            # 使用指定马赫数
            M = descent_mach
            tas = atm.mach2Tas(M, theta_val)
            cas = atm.mach2Cas(M, theta_val, delta_val, sigma_val)
            speed_mode = f"Mach {descent_mach}"
            flight_evolution = "constM"
        elif flight_level >= low_cas_fl:
            # 使用高高度CAS
            cas = conv.kt2ms(high_cas)
            tas = atm.cas2Tas(cas, delta_val, sigma_val)
            M = atm.tas2Mach(tas, theta_val)
            speed_mode = f"CAS {high_cas}kt"
            flight_evolution = "constCAS"
        else:
            # 使用低高度CAS
            cas = conv.kt2ms(low_cas)
            tas = atm.cas2Tas(cas, delta_val, sigma_val)
            M = atm.tas2Mach(tas, theta_val)
            speed_mode = f"CAS {low_cas}kt"
            flight_evolution = "constCAS"
        
        # 动态计算能量分配因子
        esf_des = Airplane.esf(
            h=altitude_m,
            DeltaTemp=DeltaTemp,
            flightEvolution=flight_evolution,
            phase="des",
            M=M
        )
        
        # 计算升力系数
        cl = bada4.CL(delta=delta_val, mass=aircraft_mass, M=M)
        
        # 计算阻力系数(干净构型)
        HLid = 0.0  # 无襟翼
        LG = "LGUP"  # 起落架收起
        cd = bada4.CD(HLid=HLid, LG=LG, CL=cl, M=M)
        
        # 计算阻力
        drag = bada4.D(delta=delta_val, M=M, CD=cd)
        
        # 计算怠速推力
        idle_thrust = bada4.Thrust(
            delta=delta_val,
            theta=theta_val,
            M=M,
            rating="LIDL",  # 怠速设置
            DeltaTemp=DeltaTemp
        )
        
        # 计算燃油流量 (kg/s)
        fuel_flow = bada4.ff(
            delta=delta_val,
            theta=theta_val,
            M=M,
            rating="LIDL",  # 怠速设置
            DeltaTemp=DeltaTemp
        )
        
        # 转换为kg/h
        fuel_flow_kgh = fuel_flow * 3600
        
        # 计算怠速下降率
        idle_descent_rate = bada4.ROCD(
            T=idle_thrust,
            D=drag,
            v=tas,
            mass=aircraft_mass,
            ESF=esf_des,
            h=altitude_m,
            DeltaTemp=DeltaTemp
        )
        
        # 计算下降角(度)
        if tas > 0:
            descent_angle = math.degrees(math.asin(idle_descent_rate / tas))
        else:
            descent_angle = 0
        
        # 计算下降梯度(百分比)
        descent_gradient = 100 * abs(idle_descent_rate / tas) if tas > 0 else 0
        
        # 计算高距离
        ft_per_nm = abs(idle_descent_rate * 196.85 / conv.ms2kt(tas) * 60) if conv.ms2kt(tas) > 0 else 0
        
        return {
            "FL": flight_level,
            "高度(ft)": altitude_ft,
            "速度模式": speed_mode,
            "马赫数": round(M, 3),
            "CAS(kt)": round(conv.ms2kt(cas), 1),
            "TAS(kt)": round(conv.ms2kt(tas), 1),
            "下降率(ft/min)": round(idle_descent_rate * 196.85, 0),
            "下降率(m/s)": round(idle_descent_rate, 2),
            "下降角(度)": round(descent_angle, 2),
            "下降梯度(%)": round(descent_gradient, 2),
            "高距离(ft/nm)": round(ft_per_nm, 0),
            "ESF": round(esf_des, 3),
            "阻力(N)": round(drag, 0),
            "怠速推力(N)": round(idle_thrust, 0),
            "燃油流量(kg/h)": round(fuel_flow_kgh, 1),
            "燃油流量(kg/s)": round(fuel_flow, 4),
            "升力系数": round(cl, 3),
            "阻力系数": round(cd, 4)
        }
    
    # 高度采样 - 每10个高度层一个点，提供更详细的数据
    flight_levels = []
    for fl in range(cruise_fl, target_fl-1, -10):
        flight_levels.append(fl)
    if target_fl not in flight_levels:
        flight_levels.append(target_fl)
    
    results = []
    for fl in flight_levels:
        result = calculate_performance(fl, aircraft_mass)
        results.append(result)
    
    # 创建DataFrame展示结果
    df = pd.DataFrame(results)
    
    # 计算累积距离和燃油消耗
    horizontal_distance = np.zeros(len(flight_levels))
    fuel_consumption = np.zeros(len(flight_levels))
    time_elapsed = np.zeros(len(flight_levels))
    
    # 主循环，集成减速段计算
    for i in range(1, len(flight_levels)):
        # 计算相邻高度层之间的高度差(英尺)
        altitude_diff = (flight_levels[i-1] - flight_levels[i]) * 100
        
        # 使用两个点的平均高距离比
        avg_ft_per_nm = (df.iloc[i-1]['高距离(ft/nm)'] + df.iloc[i]['高距离(ft/nm)']) / 2
        if avg_ft_per_nm > 0:
            segment_distance = altitude_diff / avg_ft_per_nm
        else:
            segment_distance = 0
        
        # 计算基本时间和燃油
        avg_tas_per_sec = (df.iloc[i-1]['TAS(kt)'] + df.iloc[i]['TAS(kt)']) / (2 * 3600)
        if avg_tas_per_sec > 0:
            segment_time_seconds = segment_distance / avg_tas_per_sec
        else:
            segment_time_seconds = 0
        
        avg_fuel_flow = (df.iloc[i-1]['燃油流量(kg/s)'] + df.iloc[i]['燃油流量(kg/s)']) / 2
        segment_fuel = avg_fuel_flow * segment_time_seconds
        
        # 检查是否需要减速段
        decel_distance = 0
        decel_time = 0
        decel_fuel = 0
        
        # 当前高度和下一高度
        current_fl = flight_levels[i-1]
        next_fl = flight_levels[i]
        current_cas = df.iloc[i-1]['CAS(kt)']
        
        # 检查是否穿过速度限制点
        for restriction_fl, restriction_cas in speed_restrictions:
            # 如果高度区间穿过了限制高度且当前速度超过限制
            if current_fl > restriction_fl >= next_fl and current_cas > restriction_cas:
                # 使用当前高度的燃油流量
                current_fuel_flow = df.iloc[i-1]['燃油流量(kg/s)']
                
                # 计算减速所需的距离、时间和燃油
                decel_dist, decel_t, decel_f = calculate_deceleration_segment(
                    current_cas, restriction_cas, df.iloc[i-1]['TAS(kt)'], current_fuel_flow
                )
                
                # 累加减速段的数值
                decel_distance += decel_dist
                decel_time += decel_t
                decel_fuel += decel_f
                
                # 可以选择在这里记录减速点信息
                decel_info = {
                    'FL': restriction_fl,
                    '减速前CAS': current_cas,
                    '减速后CAS': restriction_cas,
                    '减速距离(nm)': decel_dist,
                    '减速时间(s)': decel_t,
                    '减速燃油(kg)': decel_f
                }
                # decel_segments.append(decel_info)  # 可以收集所有减速段信息
        
        # 累积基本距离加减速距离
        total_segment_distance = segment_distance + decel_distance
        horizontal_distance[i] = horizontal_distance[i-1] + total_segment_distance
        
        # 累积基本时间加减速时间
        total_segment_time = segment_time_seconds + decel_time
        time_elapsed[i] = time_elapsed[i-1] + total_segment_time
        
        # 累积基本燃油加减速燃油
        total_segment_fuel = segment_fuel + decel_fuel
        fuel_consumption[i] = fuel_consumption[i-1] + total_segment_fuel
        
        # 可选：如果有减速段，在这里打印或记录减速信息
        if decel_distance > 0:
            print(f"在FL{current_fl}-FL{next_fl}之间需要减速段: "
                f"从{current_cas:.1f}kt减至{restriction_cas}kt, "
                f"需要{decel_distance:.1f}nm, {decel_time:.0f}秒, "
                f"额外燃油{decel_fuel:.1f}kg")
    
    # 添加累积数据到DataFrame
    df['累积距离(nm)'] = [round(d, 1) for d in horizontal_distance]
    df['累积时间(min)'] = [round(t/60, 1) for t in time_elapsed]
    df['累积燃油(kg)'] = [round(f, 1) for f in fuel_consumption]
    
    if print_details:
        # 打印详细表格
        print("\n详细下降数据:")
        print("-" * 150)
        
        # 选择要显示的关键列
        display_columns = [
            'FL', '速度模式', '马赫数', 'CAS(kt)', 'TAS(kt)', 
            '下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)',
            '燃油流量(kg/h)', '累积距离(nm)', '累积时间(min)', '累积燃油(kg)'
        ]
        
        # 打印表头
        header = ""
        for col in display_columns:
            if col in ['FL', '马赫数', 'CAS(kt)', 'TAS(kt)']:
                header += f"{col:>8}"
            elif col in ['速度模式']:
                header += f"{col:>12}"
            elif col in ['下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)', '燃油流量(kg/h)']:
                header += f"{col:>12}"
            else:
                header += f"{col:>14}"
        print(header)
        print("-" * 150)
        
        # 打印数据行
        for _, row in df.iterrows():
            line = ""
            for col in display_columns:
                value = row[col]
                if col in ['FL']:
                    line += f"{value:>8.0f}"
                elif col in ['马赫数']:
                    line += f"{value:>8.3f}"
                elif col in ['CAS(kt)', 'TAS(kt)']:
                    line += f"{value:>8.1f}"
                elif col in ['速度模式']:
                    line += f"{str(value):>12}"
                elif col in ['下降率(ft/min)', '高距离(ft/nm)']:
                    line += f"{value:>12.0f}"
                elif col in ['下降梯度(%)']:
                    line += f"{value:>12.2f}"
                elif col in ['燃油流量(kg/h)']:
                    line += f"{value:>12.1f}"
                else:
                    line += f"{value:>14.1f}"
            print(line)
        
        print("-" * 150)
    
    # 计算总结信息
    total_distance = horizontal_distance[-1]
    total_time = time_elapsed[-1]
    total_fuel = fuel_consumption[-1]
    total_altitude_change = (cruise_fl - target_fl) * 100
    avg_ft_per_nm = total_altitude_change / total_distance if total_distance > 0 else 0
    
    summary = {
        "Profile": profile_name,
        "Descent Distance (nm)": round(total_distance, 1),
        "Descent Time (s)": round(total_time, 0),
        "Descent Time (min)": round(total_time/60, 1),
        "Fuel Consumption (kg)": round(total_fuel, 1),
        "Average Fuel Flow (kg/h)": round(total_fuel/(total_time/3600), 1) if total_time > 0 else 0,
        "Average Descent Gradient (%)": round(total_altitude_change/6076.12/total_distance*100, 2) if total_distance > 0 else 0,
        "Altitude-to-Distance Ratio (ft/nm)": round(avg_ft_per_nm, 1)
    }
    
    if print_details:
        print(f"\n下降剖面总结:")
        print(f"总下降距离: {summary['Descent Distance (nm)']} nm")
        print(f"总下降时间: {summary['Descent Time (min)']} 分钟")
        print(f"总燃油消耗: {summary['Fuel Consumption (kg)']} kg")
        print(f"平均燃油流量: {summary['Average Fuel Flow (kg/h)']} kg/h")
        print(f"平均下降梯度: {summary['Average Descent Gradient (%)']} %")
        print(f"高距比: {summary['Altitude-to-Distance Ratio (ft/nm)']} ft/nm")
        print("=" * 80)
    
    return summary, df


In [37]:

result2_summary, result2_df = calculate_descent_profile_detailed(
    cruise_fl=390,
    target_fl=30,
    aircraft_mass= 165000,    # MLW: 172,365   B788GE70
    descent_mach=0.86,  # 这个参数在0.90 constant_cas模式下不会被使用
    high_cas=330,  # 这个参数在constant_cas模式下不会被使用
    ac_model="B788GE70",
    low_cas=250,
    low_cas_fl=100,
    print_details=True
)


详细下降剖面分析: B788GE70
飞机重量: 165.0 吨
下降剖面: 0.86M/330kt/250kt@FL100
巡航高度: FL390 (39,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL298 (29,800 ft)

详细下降数据:
------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)     累积时间(min)      累积燃油(kg)
------------------------------------------------------------------------------------------------------------------------------------------------------
     390   Mach 0.86   0.860   268.9   493.3       -2496        5.00         304      1199.9           0.0           0.0           0.0
     380   Mach 0.86   0.860   275.2   493.3       -2573        5.15         313      1168.7           3.2           0.4           7.8
     370   Mach 0.86   0.860   281.6   493.3       -2663        5.33         324      1137.1           6.4           0.8          15.1
     360   Mach

In [30]:
def add_deceleration_segments(orig_summary, orig_df, flight_levels, descent_mach, high_cas, low_cas=None, 
                              low_cas_fl=None, print_details=True):
    """
    在原始下降剖面结果上添加减速段计算
    
    参数:
    orig_summary: 原始下降剖面总结
    orig_df: 原始下降剖面详细数据
    flight_levels: 高度层列表
    descent_mach: 下降马赫数
    high_cas: 高高度CAS
    low_cas: 低高度CAS (如果不为None则添加减速段)
    low_cas_fl: 低高度CAS开始的高度层
    print_details: 是否打印详细信息
    
    返回:
    包含减速段的新总结信息和详细DataFrame
    """
    # 复制原始数据，以保持原始结果不变
    new_df = orig_df.copy()
    
    # 获取原始的累积数据
    horizontal_distance = new_df['累积距离(nm)'].values.copy()
    time_elapsed = new_df['累积时间(min)'].values.copy() * 60  # 转换为秒
    fuel_consumption = new_df['累积燃油(kg)'].values.copy()
    
    # 定义减速计算函数
    def calculate_deceleration_segment(current_cas, target_cas, fuel_flow_kg_per_sec):
        """计算减速段所需的距离、时间和燃油"""
        if current_cas <= target_cas:
            return 0, 0, 0
        
        # 根据您的要求，时间是每节1秒，距离是每节0.1海里
        decel_distance_nm = (current_cas - target_cas) / 10
        decel_time_sec = (current_cas - target_cas)  # 每节1秒
        decel_fuel = fuel_flow_kg_per_sec * decel_time_sec
        
        return decel_distance_nm, decel_time_sec, decel_fuel
    
    # 创建减速段记录列表
    decel_segments = []
    
    # 1. 添加FL100法定限速减速段(如果需要)
    fl100_index = None
    
    # 查找FL100附近的索引
    for i in range(1, len(flight_levels)):
        if flight_levels[i-1] > 100 >= flight_levels[i]:
            fl100_index = i
            break
    
    # 如果找到了FL100附近的高度段，并且速度超过250节，添加减速段
    if fl100_index is not None:
        current_cas = new_df.iloc[fl100_index-1]['CAS(kt)']
        
        if current_cas > 250:
            # 使用FL100以上一个点(通常是FL110)的燃油流量
            fuel_flow = new_df.iloc[fl100_index-1]['燃油流量(kg/s)']
            
            # 计算减速段
            decel_dist, decel_time, decel_fuel = calculate_deceleration_segment(
                current_cas, 250, fuel_flow
            )
            
            # 记录减速段信息
            decel_segments.append({
                'type': '法定减速',
                'FL': 100,
                '减速前CAS': current_cas,
                '减速后CAS': 250,
                '减速距离(nm)': decel_dist,
                '减速时间(s)': decel_time,
                '减速燃油(kg)': decel_fuel,
                'TAS(kt)': new_df.iloc[fl100_index-1]['TAS(kt)']
            })
            
            # 在FL100之后的所有点添加减速段的距离、时间和燃油
            horizontal_distance[fl100_index:] += decel_dist
            time_elapsed[fl100_index:] += decel_time
            fuel_consumption[fl100_index:] += decel_fuel
    
    # 2. 添加low_cas_fl减速段(如果有low_cas且不等于high_cas)
    # 修改第2部分的代码，避免重复计算已处理过的高度
    if low_cas is not None and low_cas_fl is not None and low_cas != high_cas:
        low_cas_index = None
        
        # 查找low_cas_fl附近的索引
        for i in range(1, len(flight_levels)):
            if flight_levels[i-1] > low_cas_fl >= flight_levels[i]:
                low_cas_index = i
                break
        
        # 只有当low_cas_fl不等于100时才进行处理
        # 或者即使是100，但法定减速没有处理（fl100_index为None）
        if low_cas_index is not None and (low_cas_fl != 100 or fl100_index is None):
            # 获取前一点的CAS
            current_cas = new_df.iloc[low_cas_index-1]['CAS(kt)']
            
            # 如果当前速度高于目标速度，添加减速段
            if current_cas > low_cas:
                # 使用low_cas_fl以上一个点的燃油流量
                fuel_flow = new_df.iloc[low_cas_index-1]['燃油流量(kg/s)']
                
                # 计算减速段
                decel_dist, decel_time, decel_fuel = calculate_deceleration_segment(
                    current_cas, low_cas, fuel_flow
                )
                
                # 记录减速段信息
                decel_segments.append({
                    'type': '剖面减速',
                    'FL': low_cas_fl,
                    '减速前CAS': current_cas,
                    '减速后CAS': low_cas,
                    '减速距离(nm)': decel_dist,
                    '减速时间(s)': decel_time,
                    '减速燃油(kg)': decel_fuel,
                    'TAS(kt)': new_df.iloc[low_cas_index-1]['TAS(kt)']
                })
                
                # 在low_cas_fl之后的所有点添加减速段的距离、时间和燃油
                horizontal_distance[low_cas_index:] += decel_dist
                time_elapsed[low_cas_index:] += decel_time
                fuel_consumption[low_cas_index:] += decel_fuel
    # 在现有的代码中的decel_segments处理部分后面添加:

    # 3. 添加FL30最终进近减速段
    fl030_index = None

    # 查找FL30附近的索引
    for i in range(1, len(flight_levels)):
        if flight_levels[i-1] > 30 >= flight_levels[i]:
            fl030_index = i
            break

    # 如果找到了FL30附近的高度段，添加从250减速到220的段
    if fl030_index is not None:
        # 获取前一点的CAS（通常应该是250节，如果经过了FL100减速）
        current_cas = new_df.iloc[fl030_index-1]['CAS(kt)']
        
        # 如果当前速度高于220节，添加减速段
        if current_cas > 220:
            # 使用FL30以上一个点的燃油流量
            fuel_flow = new_df.iloc[fl030_index-1]['燃油流量(kg/s)']
            
            # 计算减速段
            decel_dist, decel_time, decel_fuel = calculate_deceleration_segment(
                current_cas, 220, fuel_flow
            )
            
            # 记录减速段信息
            decel_segments.append({
                'type': '进近减速',
                'FL': 30,
                '减速前CAS': current_cas,
                '减速后CAS': 220,
                '减速距离(nm)': decel_dist,
                '减速时间(s)': decel_time,
                '减速燃油(kg)': decel_fuel,
                'TAS(kt)': new_df.iloc[fl030_index-1]['TAS(kt)']
            })
            
            # 在FL30之后的所有点添加减速段的距离、时间和燃油
            horizontal_distance[fl030_index:] += decel_dist
            time_elapsed[fl030_index:] += decel_time
            fuel_consumption[fl030_index:] += decel_fuel
    
    # 更新DataFrame中的累积数据
    new_df['累积距离(nm)'] = [round(d, 1) for d in horizontal_distance]
    new_df['累积时间(min)'] = [round(t/60, 1) for t in time_elapsed]
    new_df['累积燃油(kg)'] = [round(f, 1) for f in fuel_consumption]
    
    # 打印减速段信息
    if print_details and decel_segments:
        print("\n减速段分析:")
        print("-" * 120)
        print(f"{'类型':<12}{'高度层':<10}{'减速前CAS':<12}{'减速后CAS':<12}{'距离(nm)':<12}{'时间(s)':<10}{'燃油(kg)':<12}{'TAS(kt)':<10}")
        print("-" * 120)
        
        for segment in decel_segments:
            print(f"{segment['type']:<12}FL{segment['FL']:<8}{segment['减速前CAS']:<12.1f}"
                  f"{segment['减速后CAS']:<12.1f}{segment['减速距离(nm)']:<12.1f}"
                  f"{segment['减速时间(s)']:<10.0f}{segment['减速燃油(kg)']:<12.1f}{segment['TAS(kt)']:<10.1f}")
        
        print("-" * 120)
    
    if print_details:
        # 打印详细表格
        print("\n添加减速段后的详细下降数据:")
        print("-" * 165)
        
        # 选择要显示的关键列
        display_columns = [
            'FL', '速度模式', '马赫数', 'CAS(kt)', 'TAS(kt)', 
            '下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)',
            '燃油流量(kg/h)', '累积距离(nm)', '累积时间(min)', '累积燃油(kg)'
        ]
        
        # 打印表头
        header = ""
        for col in display_columns:
            if col in ['FL', '马赫数', 'CAS(kt)', 'TAS(kt)']:
                header += f"{col:>8}"
            elif col in ['速度模式']:
                header += f"{col:>12}"
            elif col in ['下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)', '燃油流量(kg/h)']:
                header += f"{col:>12}"
            else:
                header += f"{col:>14}"
        print(header)
        print("-" * 165)
        
        # 打印数据行
        for _, row in new_df.iterrows():
            line = ""
            for col in display_columns:
                # 对于最后一个TAS(kt)列，使用相同的TAS(kt)值
                value = row['TAS(kt)'] if col == 'TAS(kt)' and display_columns.index(col) == len(display_columns) - 1 else row[col]
                if col in ['FL']:
                    line += f"{value:>8.0f}"
                elif col in ['马赫数']:
                    line += f"{value:>8.3f}"
                elif col in ['CAS(kt)', 'TAS(kt)']:
                    line += f"{value:>8.1f}"
                elif col in ['速度模式']:
                    line += f"{str(value):>12}"
                elif col in ['下降率(ft/min)', '高距离(ft/nm)']:
                    line += f"{value:>12.0f}"
                elif col in ['下降梯度(%)']:
                    line += f"{value:>12.2f}"
                elif col in ['燃油流量(kg/h)']:
                    line += f"{value:>12.1f}"
                else:
                    line += f"{value:>14.1f}"
            print(line)
        
        print("-" * 165)
    
    # 计算新的总结信息
    total_distance = horizontal_distance[-1]
    total_time = time_elapsed[-1]
    total_fuel = fuel_consumption[-1]
    total_altitude_change = (flight_levels[0] - flight_levels[-1]) * 100
    avg_ft_per_nm = total_altitude_change / total_distance if total_distance > 0 else 0
    
    new_summary = {
        "Profile": orig_summary["Profile"] + " (含减速段)",
        "Descent Distance (nm)": round(total_distance, 1),
        "Descent Time (s)": round(total_time, 0),
        "Descent Time (min)": round(total_time/60, 1),
        "Fuel Consumption (kg)": round(total_fuel, 1),
        "Average Fuel Flow (kg/h)": round(total_fuel/(total_time/3600), 1) if total_time > 0 else 0,
        "Average Descent Gradient (%)": round(total_altitude_change/6076.12/total_distance*100, 2) if total_distance > 0 else 0,
        "Altitude-to-Distance Ratio (ft/nm)": round(avg_ft_per_nm, 1)
    }
    
    # 打印新的总结信息
    if print_details:
        print(f"\n添加减速段后的下降剖面总结:")
        print(f"总下降距离: {new_summary['Descent Distance (nm)']} nm (增加{new_summary['Descent Distance (nm)']-orig_summary['Descent Distance (nm)']:.1f}nm)")
        print(f"总下降时间: {new_summary['Descent Time (min)']} 分钟 (增加{new_summary['Descent Time (min)']-orig_summary['Descent Time (min)']:.1f}分钟)")
        print(f"总燃油消耗: {new_summary['Fuel Consumption (kg)']} kg (增加{new_summary['Fuel Consumption (kg)']-orig_summary['Fuel Consumption (kg)']:.1f}kg)")
        print(f"平均下降梯度: {new_summary['Average Descent Gradient (%)']} %")
        print(f"高距比: {new_summary['Altitude-to-Distance Ratio (ft/nm)']} ft/nm")
        print("=" * 80)
    
    return new_summary, new_df, decel_segments

In [31]:
# 计算不包含减速段的原始下降剖面
orig_summary, orig_df = calculate_descent_profile_detailed(
    cruise_fl=370,
    target_fl=30,
    aircraft_mass=60000,
    descent_mach=0.73,
    high_cas=250,
    ac_model="A320-232",
    low_cas=220,
    low_cas_fl=150,
    print_details=True
)

# 在原始结果上添加减速段
print("\n添加减速段的计算...")
new_summary, new_df, decel_segments = add_deceleration_segments(
    orig_summary=orig_summary,
    orig_df=orig_df,
    flight_levels=list(orig_df['FL']),
    descent_mach=0.73,
    high_cas=250,
    low_cas=220,
    low_cas_fl=150,
    print_details=True
)


详细下降剖面分析: A320-232
飞机重量: 60.0 吨
下降剖面: 0.73M/250kt/220kt@FL150
巡航高度: FL370 (37,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL342 (34,200 ft)

详细下降数据:
------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)     累积时间(min)      累积燃油(kg)
------------------------------------------------------------------------------------------------------------------------------------------------------
     370   Mach 0.73   0.730   234.7   418.7       -2234        5.27         320       535.0           0.0           0.0           0.0
     360   Mach 0.73   0.730   240.3   418.9       -2425        5.72         347       533.3           3.0           0.4           3.8
     350   Mach 0.73   0.730   245.9   420.8       -2475        5.81         353       531.4           5.9           0.8           7.4
     340   CAS 2

In [38]:
# 计算不包含减速段的原始下降剖面
orig_summary, orig_df = calculate_descent_profile_detailed(
    cruise_fl=370,
    target_fl=30,
    aircraft_mass=60000,
    descent_mach=0.80,
    high_cas=310,
    ac_model="A320-232",
    low_cas=250,
    low_cas_fl=100,
    print_details=True
)

# 在原始结果上添加减速段
print("\n添加减速段的计算...")
new_summary, new_df, decel_segments = add_deceleration_segments(
    orig_summary=orig_summary,
    orig_df=orig_df,
    flight_levels=list(orig_df['FL']),
    descent_mach=0.80,
    high_cas=310,
    low_cas=250,
    low_cas_fl=100,
    print_details=True
)


详细下降剖面分析: A320-232
飞机重量: 60.0 吨
下降剖面: 0.8M/310kt/250kt@FL100
巡航高度: FL370 (37,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL290 (29,000 ft)

详细下降数据:
------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)     累积时间(min)      累积燃油(kg)
------------------------------------------------------------------------------------------------------------------------------------------------------
     370    Mach 0.8   0.800   259.7   458.9       -2748        5.91         359       560.3           0.0           0.0           0.0
     360    Mach 0.8   0.800   265.8   459.0       -3072        6.61         401       556.6           2.6           0.3           3.2
     350    Mach 0.8   0.800   271.9   461.1       -3176        6.80         413       552.4           5.1           0.7           6.2
     340    Mach 

飞机类型: B788GE70  
最大起飞重量 (MTOW): 227,930 kg
最大着陆重量 (MLW): 172,365 kg
参考面积: 360.5 m²
空重: 111,493 kg
最大零燃料重量: 161,025 kg

In [ ]:
# 计算不包含减速段的原始下降剖面
orig_summary, orig_df = calculate_descent_profile_detailed(
    cruise_fl=390,
    target_fl=30,
    aircraft_mass=60000,
    descent_mach=0.73,
    high_cas=250,
    ac_model="A320-232",
    low_cas=220,
    low_cas_fl=150,
    print_details=True
)

# 在原始结果上添加减速段
print("\n添加减速段的计算...")
new_summary, new_df, decel_segments = add_deceleration_segments(
    orig_summary=orig_summary,
    orig_df=orig_df,
    flight_levels=list(orig_df['FL']),
    descent_mach=0.73,
    high_cas=250,
    low_cas=220,
    low_cas_fl=150,
    print_details=True
)